In [1]:
from pyspark import SparkContext, SparkConf 

In [2]:
# pyspark 연결하기 (SparkSession이 아닌, SparkContext로 연결하는 고전적인 방법임.)
conf = SparkConf().setAppName("RDD_practice").setMaster("local[*]")
sc = SparkContext(conf=conf)
print(sc)

23/11/26 11:10:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local[*] appName=RDD_practice>


# Part1: RDD 생성 및 기본 연산

# RDD를 생성하는 두 가지 방법:

1. 드라이버 프로그램에서 기존의 컬렉션을 Parallelizing 하는 방법
2. 공유 파일 시스템, HDFS, HBase, Hadoop InputFormat을 제공하는 데이터 소스 등과 같은 외부 데이터 저장소를 참조하는 방법

In [56]:
# 랜덤 데이터 생성
import random
random_list = random.sample(range(0, 40), 10)
print(random_list)

[7, 37, 21, 22, 6, 17, 15, 28, 38, 30]


In [57]:
# RDD 생성
rdd1 = sc.parallelize(random_list, 4)
rdd1.collect()

[7, 37, 21, 22, 6, 17, 15, 28, 38, 30]

In [58]:
sc.parallelize?

Signature: sc.parallelize(c, numSlices=None)
Docstring:
Distribute a local Python collection to form an RDD. Using xrange
is recommended if the input represents a range for performance.

>>> sc.parallelize([0, 2, 3, 4, 6], 5).glom().collect()
[[0], [2], [3], [4], [6]]
>>> sc.parallelize(xrange(0, 6, 2), 5).glom().collect()
[[], [0], [], [2], [4]]
File:      /usr/local/lib/python3.9/dist-packages/pyspark/context.py
Type:      method

In [59]:
# 전체 파티션 수 조회
print(rdd1.getNumPartitions())

# 파티션별 분산된 데이터 조회
print(rdd1.glom().collect())

# 파티션별 분산된 데이터 중 2개의 파티션만 보기
print("The two partitions", rdd1.glom().take(2))

# 마지막 파티션 출력
print("The last partition", rdd1.glom().collect()[-1])

4
[[7, 37], [21, 22], [6, 17], [15, 28, 38, 30]]
The two partitions [[7, 37], [21, 22]]
The last partition [15, 28, 38, 30]


In [22]:
# count() : RDD 전체 element 개수
rdd1.count()

10

In [24]:
# first() : 첫 번쨰 파티션의 첫 번째 element 값
rdd1.first()

4

In [33]:
# top(): 값이 높은 상위 N개의 elemenet 출력
# drive 프로세스에 모든 element를 보낸 후, 정렬(sort) 후 값을 반환하므로 데이터의 사이즈가 크면 사용에 주의가 필요하다.
print("Top 1 :", rdd1.top(1))
print("Top 2 :", rdd1.top(2))

Top 1 : [38]
Top 2 : [38, 37]


In [35]:
# distinct() : 중복을 제거한 element 반환
rdd1.distinct().collect()

[4, 20, 37, 33, 5, 30, 34, 38, 26, 19]

In [46]:
def myfunc(item):
    return (item+1) * 3

# map(): 각 elemnt에 function을 적용한 결과를 RDD로 반환 (파티션의 수는 변하지 않는다.)
rdd_map = rdd1.map(myfunc)
print("def function map :", rdd_map.glom().collect())

# map의 인자로 lambda(익명함수)도 사용 가능하다.
rdd_map = rdd1.map(lambda item: (item+1) * 3)
print("lambda function map :", rdd_map.glom().collect())

def function map : [[15, 63], [60, 114], [102, 93], [18, 105, 117, 81]]
lambda function map : [[15, 63], [60, 114], [102, 93], [18, 105, 117, 81]]


In [66]:
# filter(): element를 조건(function)에 부합하는 element만 필터링하여 RDD 반환 (파티션의 수는 변하지 않는다. 따라서, empty한 파티션이 발생 가능하다.)
# empty한 파티션은 GC(Garbage Collect) 기술인 repartition/coalese릍 통해 차후 제거 가능하다.
rdd_filter = rdd1.filter(lambda x: x%3==0)
print(rdd_filter.glom().collect())

print("After filtering count :", rdd_filter.count())
print("Repartition :", rdd_filter.repartition(1).glom().collect())

[[], [21], [6], [15, 30]]
After filtering count : 4
Repartition : [[21, 6, 15, 30]]


In [102]:
# flatMap(): 파티션별 모든 element를 하나의 단일 컬렉션으로 반환한다. (파티션 수가 변하는 것이 아니다. 파티션 내부의 차원을 한 단계 flat하게 만든다.)
rdd_flatmap = rdd1.flatMap(lambda x: [x+2, x+5])

# map()과 비교하기
rdd_map = rdd1.map(lambda x: [x+2, x+5])

# 비교 시작
print("flatMap Collect: ", rdd_flatmap.collect())
print("map Collect: ", rdd_map.collect(), "\n")

print("flatMap glom collect", rdd_flatmap.glom().collect())
print("map glom collect: ", rdd_map.glom().collect(), "\n")

print("flatMap Partition Num:", rdd_flatmap.getNumPartitions())
print("map Partition Num:", rdd_map.getNumPartitions())

flatMap Collect:  [9, 12, 39, 42, 23, 26, 24, 27, 8, 11, 19, 22, 17, 20, 30, 33, 40, 43, 32, 35]
map Collect:  [[9, 12], [39, 42], [23, 26], [24, 27], [8, 11], [19, 22], [17, 20], [30, 33], [40, 43], [32, 35]] 

flatMap glom collect [[9, 12, 39, 42], [23, 26, 24, 27], [8, 11, 19, 22], [17, 20, 30, 33, 40, 43, 32, 35]]
map glom collect:  [[[9, 12], [39, 42]], [[23, 26], [24, 27]], [[8, 11], [19, 22]], [[17, 20], [30, 33], [40, 43], [32, 35]]] 

flatMap Partition Num: 4
map Partition Num: 4


In [111]:
# reduce(): 각 파티션별 배열(1차원)의 element에 대해 function을 수행한다. 파티션별 function의 return 값에 대해 파티션 전체에 대한 reduce를 계속 실행한다.
# reduce()는 transformation이 아닌, action 함수이다.

# flatMap glom collect: [[9, 12, 39, 42], [23, 26, 24, 27], [8, 11, 19, 22], [17, 20, 30, 33, 40, 43, 32, 35]] 라면,
# step1 : [((9+12)+39)+42], [((23+26)+24)+27], [((8+11)+19)+22], [((((((17+20)+30)+33)+40)+43)+32)+35]]
# step2 : (([102] + [100]) + [60]) + [250]
# Return : 512

rdd_flatmap.reduce(lambda x,y: x+y)

512

In [114]:
# 기술 통계 (Descriptive statistic)
print([rdd1.max(), rdd1.min(), rdd1.mean(), round(rdd1.stdev(), 2), rdd1.sum()])

[38, 6, 22.1, 10.66, 221]


In [130]:
# mapPartitions(): 파티션별 function을 하여 특정 결과를 반환한다. action에 reduce가 있다면, transformation에는 mapPartition이라고 생각하면 쉽다.
# 각 파티션의 element에 대해 fuct을 적용한 중간 결과(yield)를 다음 element의 func 연산을 할 떄 사용하므로, 제너레이터에 사용하는 yield를 사용해야 한다!

def myfunc(partition):
    sumatition = 0
    
    for item in partition:
        sumatition = sumatition + item
        
    yield sumatition

print("rdd1 glom collect:", rdd1.glom().collect())
print("rdd1 mapPartition:", rdd1.mapPartitions(myfunc).collect())
print("rdd1 glom Partition:", rdd1.mapPartitions(myfunc).glom().collect())

rdd1 glom collect: [[7, 37], [21, 22], [6, 17], [15, 28, 38, 30]]
rdd1 mapPartition: [44, 43, 23, 111]
rdd1 glom Partition: [[44], [43], [23], [111]]


# Part2: Advanced RDD Transformations and Actions

In [149]:
# union(): 두 RDD를 결합하여 새로운 RDD를 반환한다. 결합한 RDD의 파티션 수는 결합에 사용된 RDD의 파티션의 총 합과 같다.

print("RDD 1:", rdd1.collect())

rdd2 = sc.parallelize([1, 14, 37, 20, 28, 10, 13, 3], 2)
print("RDD 2:", rdd2.collect())

rdd_union = rdd1.union(rdd2)
print("RDD union:", rdd_union.collect())

print("RDD Partition Num:", rdd_union.getNumPartitions())

RDD 1: [7, 37, 21, 22, 6, 17, 15, 28, 38, 30]
RDD 2: [1, 14, 37, 20, 28, 10, 13, 3]
RDD union: [7, 37, 21, 22, 6, 17, 15, 28, 38, 30, 1, 14, 37, 20, 28, 10, 13, 3]
RDD Partition Num: 6


In [153]:
# intersection(): 두 RDD의 공통된 element만 추출하여 새로운 RDD를 반환한다. 교차시킨 RDD의 파티션 수는 교차에 사용된 RDD의 파티션의 총 합과 같다.
rdd_intersection = rdd1.intersection(rdd2)

print("RDD intersection:", rdd_intersection.collect())
print("RDD intersection glom collect:" , rdd_intersection.glom().collect())
print("RDD Partition Num:", rdd_intersection.getNumPartitions())

RDD intersection: [37, 28]
RDD intersection glom collect: [[], [37], [], [], [28], []]
RDD Partition Num: 6


In [157]:
# Find empty partitions

counter = 0

for item in rdd_intersection.glom().collect():
    if len(item) == 0:
        counter += 1

counter

4

In [159]:
# coalesce(numPartitions): 파티션의 수를 줄인다. 
rdd_intersection.coalesce(1).glom().collect()

[[37, 28]]

In [169]:
# takeSample(withReplacement, num, [seed]): 무작위로 데이터를 추출하여 반환한다.
# collect()와 마찬가지로, parallel 하지 않고 drive 메모리에 모든 element를 모아 랜덤 추출하기 때문에 스파크 클러스터가 손상 될 수 있어 사용에 주의가 필요하다.
rdd1.takeSample(False, 5)

[22, 17, 28, 30, 21]